In [1]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import helperfunc as hf

import trainers as t
%load_ext autotime

time: 251 µs (started: 2021-02-28 10:47:21 -05:00)


In [2]:
##load data
In = np.load('data/nn_output_10.npy')
out = np.load('data/nn_input_10.npy')
N,Dx = np.shape(In)
N,Dy =np.shape(out)

print('Input shape: ', np.shape(In))
print('Output shape: ',np.shape(out))

## Normalizing
for i in range(Dx):
    if np.min(In[:,i])<-200:
        new_data = (In[:,i]-np.mean(In[:,i]))/(np.std(In[:,i]))
        In[:,i]=new_data

Input shape:  (1000000, 24)
Output shape:  (1000000, 27)
time: 570 ms (started: 2021-02-28 10:47:21 -05:00)


In [3]:
from sklearn.model_selection import train_test_split
In, X_test, out, y_test = train_test_split(In, out, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)


time: 826 ms (started: 2021-02-28 10:47:21 -05:00)


In [4]:
config_single ={
         "lr":1e-3,
         "epochs": 10,
         "batch_size": 32,
         "iters": 100,#int(N/32),
         "input_dim" : Dx,
         "output_dim" : Dy,
         "hidden1" :700,
         "hidden2" :700,
         "display_interval":20,
         "print_it" : True,
    }
NNl, critl,mselosst,train_loss,val_loss=t.trainsingle(config_single,In,out,X_val,y_val)

print('Single Network Loss: ',mselosst)

starting training
iter :  0 	 train loss :1.366


RuntimeError: Tensor for argument #2 'mat1' is on CPU, but expected it to be on GPU (while checking arguments for addmm)

time: 2 s (started: 2021-02-28 10:47:24 -05:00)


In [6]:
import torch

PATH="single_dict_model.pt"
torch.save(NNl.state_dict(), PATH)


f, (ax1, ax2, ax3) = plt.subplots(1, 3,figsize=(16,12))
ax1.plot(train_loss)
ax1.set_title("Training Loss")
ax1.set_xlabel("No. of epochs")
ax1.set_ylabel("MSE")


ax2.plot(val_loss)
ax2.set_title("Validation loss")
ax2.set_xlabel("No. of epochs")
ax2.set_ylabel("MSE")

ax3.plot(train_loss)
ax3.plot(val_loss)
ax3.set_title("Training and valiation loss")
ax3.set_xlabel("No. of epochs")
ax3.set_ylabel("MSE")


NameError: name 'NNl' is not defined

time: 9.38 ms (started: 2021-02-28 10:47:41 -05:00)


In [7]:
#cor = hf.corr(In,out,Dx,Dy)
#print(np.argmin(cor))
config_section ={
        "init_select":7,
        "runs":1,
         "lr":1e-3,
         "epochs": 100,
         "batch_size": 32,
         "iters": 50,#int(N/32),
         "input_dim" : Dx,
         "output_dim" : Dy,
         "hidden1" :100,
         "hidden2" :100,
         "display_interval":30,
         "print_it" : True,
         "c":50,
         "ref_s": 7,#int(np.ceil(Dx/Dy)),
         "ref_v":np.var(In),
         "phi_1":1.0,
         "phi_2":1e-3,
         "phi_3":1e-6,
         "beta":0.5,
         "dim":0#np.argmin(cor)
    }

time: 71.8 ms (started: 2021-02-28 10:47:44 -05:00)


In [8]:
SN = config_section["init_select"]
iters = config_section["runs"]
dim = config_section["dim"]

l=np.zeros((iters,4))

sns=np.zeros(iters)
on,bn = hf.init_s(SN,dim,In)
dmse = np.zeros(iters)
ds = np.zeros(iters)
dv = np.zeros(iters)

mse_best=10000
best_config={}

for i in range(iters):
    NN, crit,all_in,all_out,lossind,train_loss,test_loss = t.trainsect(config_section, In, out,SN,on,bn,X_val,y_val)

    l[i,:]=hf.loss_1(all_in,all_out,SN,bn,on,config_section,NN,lossind,mselosst)
    
    bold,bnew,dmse[i],dv[i],ds[i] =hf.update_b(In,all_in,all_out,bn,on,SN,NN,config_section,lossind,mselosst,thresh=0.2)
    
    SN1,bn,on =hf.gen_s_b_o(bold,bnew,on,config_section,In)
    if mse_best >=np.mean(lossind):
        mse_best=np.mean(lossind)
        best_config['NN']=NN
        best_config['o']=on
        best_config['b']=bn
        best_config['sn']=SN
        best_config['crit']=crit
        best_config['all_in']=all_in
        best_config['all_out']=all_out
    print('Sn: ',SN,' bnew: ',bn, 'bold: ',bold, '\n \t LOSS2: ',np.mean(lossind),'\n mse:',l[i,1],'varloss:',l[i,2],'sloss:',l[i,3])
    sns[i]=SN
    SN=SN1
    

sorting data for section:  0
starting training for section:  0
iter :  0 	 train loss :9.474


/home/labstation/Downloads/DivNet/helperfunc.py:6: RuntimeWarning: overflow encountered in exp
  return 1/(1+ np.exp(slope*(data-(offset+band))))


iter :  30 	 train loss :0.020
iter :  60 	 train loss :0.014
iter :  90 	 train loss :0.011


RuntimeError: Tensor for argument #2 'mat1' is on CPU, but expected it to be on GPU (while checking arguments for addmm)

time: 12.5 s (started: 2021-02-28 10:47:46 -05:00)


In [ ]:
print('dmse',dmse)
print('dv',dv)
print('ds',ds)

In [ ]:
#### saving best model and plotting progress

plt.plot(l[:,0],'b-x')
plt.plot(l[:,1],'g-x')
plt.plot(l[:,2],'r-x')
plt.plot(l[:,3],'y-x')
plt.legend(['l1', 'mse', 'var_loss','sloss'])


NN=best_config['NN']
on=best_config['o']
bn=best_config['b']
SN=best_config['sn']
crit=best_config['crit']
all_in=best_config['all_in']
all_out=best_config['all_out']

for i in range(SN):
    PATH="section_dict_model"+str(i)+".pt"
    torch.save(NN[str(i)].state_dict(), PATH)


In [ ]:
config_tiny ={
        "input_dim":1,
        "output_dim":1,
        "hidden1":2,
        "hidden2":2,
        "lr":1e-3,
        "epochs": 200,
        "batch_size": 500,
        "iters": 100,#int(N/500),
        "display_interval":10,
        "print_it":True,
        "dim":config_section["dim"]
}
dim = config_tiny["dim"]
data,labels= hf.generate_rdata(SN,on,bn,In,dim)
val_data,val_labels =hf.generate_rdata(SN,on,bn,X_val,dim)
test_data,test_labels =hf.generate_rdata(SN,on,bn,X_test,dim)

NNt, criteriont,train_loss,val_loss=t.traintiny(config_tiny, data,labels,val_data,val_labels)


In [ ]:
PATH="tiny_dict_model.pt"
torch.save(NNt.state_dict(), PATH)

In [ ]:
import torch
NNt.eval()
test_data = torch.tensor(test_data, dtype=torch.float) #  tensor
outputs = NNt(test_data)
outs=np.round(outputs.detach().numpy())
tot=np.abs(outs-test_labels)
tot= np.sum(tot)
print('Incorrect labels', tot)

In [ ]:
import torch.nn as nn
def test(NNt,NN,X_test, y_test,SN,dim,on,bn):
    criterion = nn.MSELoss()
    NNt.eval()
    N,D = np.shape(X_test)
    data =np.reshape(X_test[:,dim],(N,1))
    data = torch.tensor(data, dtype=torch.float)
    X_test = torch.tensor(X_test, dtype=torch.float,requires_grad=True)
    y_test = torch.tensor(y_test, dtype=torch.float) #  tensor
    selectors = NNt(data).detach().numpy()
    loss = 0 
    print('calculating loss')
    for i in range(N):
        j = int(selectors[i])
        NN[str(j)].eval()
        ypred=NN[str(j)](X_test[i])
        loss += criterion(ypred, y_test[i]).item()
        #ypred[i]=NN[str(j)](torch.tensor(X_test[i], dtype=torch.float)).detach().numpy()
    #ypred = torch.tensor(ypred, dtype=torch.float) #  tensor
    
    return loss/N

In [ ]:
Nt,Dt = np.shape(X_test)

In [ ]:
dim = config_tiny["dim"]

loss = test(NNt,NN,X_test,y_test,SN,dim,on,bn)
print('test loss =', loss)

In [ ]:
print('time per data point:',74000/Nt)

In [ ]:
dim = config_tiny["dim"]
loss = test(NNt,NN,In,out,SN,dim,on,bn)
print('train loss =', loss)

In [ ]:
Nt,Dt = np.shape(In)

print('time per data point:',156000/Nt)

In [ ]:
import torch.nn as nn
def testl(NNl,X_test, y_test):
    criterion = nn.MSELoss()
    N,D = np.shape(X_test)

    X_test = torch.tensor(X_test, dtype=torch.float,requires_grad=True)
    y_test = torch.tensor(y_test, dtype=torch.float) #  tensor
    loss = 0 
    print('calculating loss')
    NNl.eval
    for i in range(N):
        NNl.eval()
        ypred=NNl(X_test[i])
        loss += criterion(ypred, y_test[i]).item()
        #ypred[i]=NN[str(j)](torch.tensor(X_test[i], dtype=torch.float)).detach().numpy()
    #ypred = torch.tensor(ypred, dtype=torch.float) #  tensor
    
    return loss/N

In [ ]:
dim = config_tiny["dim"]

loss = testl(NNl,X_test,y_test)
print('test loss =', loss)

In [ ]:
Nt,Dt = np.shape(X_test)

print('time per data point:',79000/Nt)

In [ ]:
dim = config_tiny["dim"]
loss = testl(NNl,In,out)
print('train loss =', loss)

In [ ]:
Nt,Dt = np.shape(In)

print('time per data point:',155000/Nt)